In [3]:
import pandas as pd
import numpy as np
import imageio
from PIL import Image
import myfunctions
from myfunctions import event_separation

csvfile = r'C:\Users\roumba\Documents\Software\deep-events\Videos\21.09_mtstaygold_cos7_iSIM_1\mtStayGold__1_points.csv'
imagefile= r'C:\Users\roumba\Documents\Software\deep-events\Videos\21.09_mtstaygold_cos7_iSIM_1\mtStayGold__1_MMStack_Pos0.ome.tif'
img = Image.open(imagefile)
datacsv = pd.read_csv(csvfile)
list1=event_separation(datacsv)
l=len(list1)



In [11]:
def image_crop_negative(l,list_of_divisions, data, img):
    division_list=[]
    for index_list in range(0, l):
        l1=len(list_of_divisions[index_list])
        division_list=[]

        for index_list1 in range(0,l1):
            division_list.append(list_of_divisions[index_list][index_list1])
            minlist=division_list[0]
            maxlist=division_list[index_list1]
            if minlist>9 and maxlist<90:

                first_frame_before=minlist-10               #frame numbers back and forth
                last_frame_after=maxlist+10

                data_croped_before_event= data.iloc[first_frame_before:minlist,1:4]
                data_croped_after_event= data.iloc[maxlist:last_frame_after,1:4]

                frame1_before=int(data_croped_before_event['axis-0'].min())
                frame2_before=int(data_croped_before_event['axis-0'].max())
                frame1_after=int(data_croped_after_event['axis-0'].min())
                frame2_after=int(data_croped_after_event['axis-0'].max())

                ymean_before = data_croped_before_event['axis-1'].mean()
                xmean_before = data_croped_before_event['axis-2'].mean()
                ycrop1_b=ymean_before+128
                ycrop2_b=ymean_before-128
                xcrop1_b=xmean_before+128
                xcrop2_b=xmean_before-128
                dataar_b=np.zeros((frame2_before-frame1_before, 256, 256))

                ymean_after = data_croped_after_event['axis-1'].mean()
                xmean_after = data_croped_after_event['axis-2'].mean()
                ycrop1_a=ymean_after+128
                ycrop2_a=ymean_after-128
                xcrop1_a=xmean_after+128
                xcrop2_a=xmean_after-128
                dataar_a=np.zeros((frame2_after-frame1_after, 256, 256))

                for frame_index_b, frame_number_b in enumerate(range (frame1_before, frame2_before)):
                    img.seek(frame_number_b) #starts from 0 I think?
                    box_b = (xcrop2_b, ycrop2_b, xcrop1_b, ycrop1_b) #choose dimensions of box
                    imcrop= img.crop(box_b)
                    
                    dataar_b[frame_index_b, :, :] = np.array(imcrop)

                for frame_index_a, frame_number_a in enumerate(range (frame1_after, frame2_after)):
                    img.seek(frame_number_a) #starts from 0 I think?
                    box_a = (xcrop2_a, ycrop2_a, xcrop1_a, ycrop1_a) #choose dimensions of box
                    imcrop= img.crop(box_a)
                    
                    dataar_a[frame_index_a, :, :] = np.array(imcrop)
            
                currname_crop_b = f'image_{index_list}_b.tiff'
                currname_crop_a = f'image_{index_list}_a.tiff'
                imageio.mimwrite(currname_crop_b, (dataar_b).astype(np.uint16))   
                imageio.mimwrite(currname_crop_a, (dataar_a).astype(np.uint16))


In [12]:
image_crop_negative(l,list1, datacsv, img)